## basic functions: 
* this file can generate some `.md` files that are convenient for reading.
* some operations that can be useful when you decide to use `MySQL` or `MongoDB`.

In [1]:
import json
with open("families_with_all_info.json", "r") as fwai:
    families = json.load(fwai)

In [2]:
for i in families:
    print(i)

Wiki-words
bert
biggan
biggan-deep
compare_gan-model
compare_gan-s3gan
i3d
image_augmentation
image_augmentation-nas
imagenet-amoebanet
imagenet-inception_resnet_v2
imagenet-inception_v1
imagenet-inception_v2
imagenet-inception_v3
imagenet-mobilenet_v1
imagenet-mobilenet_v2
imagenet-nasnet
imagenet-pnasnet
imagenet-resnet_v1
imagenet-resnet_v2
nnlm
quantops
random-nnlm
tf2-preview-gnews-swivel
tf2-preview-inception_v3
tf2-preview-mobilenet_v2
tf2-preview-nnlm
universal-sentence-encoder
universal-sentence-encoder-xling


In [2]:
from pprint import pprint

def regroup_family_by_application_domain(diction):
    new_families = {}
    try:
        for family in diction:
            for module_name in diction[family]:
                if "description" in module_name:
                    continue
                module = diction[family][module_name]
                application_domain = module["application_domain"]
                family_description = diction[family]["family_description"]
                # print(application_domain, family_description)
                if application_domain not in new_families:
                    new_families[application_domain] = {
                        family: {
                            "family_description": family_description,
                            module_name: module
                        }
                    }
                else:
                    # new_families_level1 = new_families[application_domain]
                    if family not in new_families[application_domain]:
                        new_families[application_domain][family] = {
                            "family_description": family_description,
                            module_name: module
                        }
                    else:
                        new_families[application_domain][family][module_name] = module
    except:
        print(module_name)
    # with open("heheda.json", "w") as hhd:
    #     json.dump(new_families, hhd, sort_keys=True, indent=4)
    return new_families

new_families = regroup_family_by_application_domain(families)

## turn into .md files. 

In [53]:
# with open("lets_see_modules_in_families.md", "w")
def write_md_files(new_families): 
    import os

    md_dir = os.path.join(os.getcwd(), "MD_File")
    if not os.path.exists(md_dir):
        os.makedirs(md_dir)

    for application_domain in new_families:
        with open(os.path.join(md_dir, "{}.md".format(application_domain)), "w") as ad:
            app_families = new_families[application_domain]
            family_count = 1
            for family_name, family in app_families.items():
                ad.write("## {}. {}\n\n".format(family_count, family_name.replace("_", "\_")))
                module_count = 1
                for module_name, module in family.items():
                    if "description" in module_name:
                        ad.write("**{}**\n\n".format(module))
                        continue

                    ##############################
                    ## changelog transform
                    new_cl = "    None"
                    if module["changelog"] != None:
                        new_cl = "\n\n".join(["    {}".format(i) for i in module["changelog"].split("\n")])
                    #############################
                    content = """### {}.{} [{}@{}]({})\n\n
    Module Description: \n\n{}\n\n
    References: \n\n{}\n\n
    Changelog: \n\n{}\n\n""".format(
                                family_count, module_count, module_name.replace("_", "\_"), module["version"], module["link"], 
                                "\n\n".join(["* {}".format(i) for i in module["module_description"].split(";")]), 
                                "" if module["references"] == None else "\n\n".join(["* *{}*".format(i) for i in module["references"]]), 
                                new_cl #module["changelog"]
                            )
                    ad.write(content)
                    module_count += 1
                family_count += 1

write_md_files(new_families)

## MYSQL operation:

## MongoDB operation: 

If you want to use MongoDB through python, you have to do 2 things: 
* install pymongo
    * ! pip install pymongo
* install and start the MongoDB service. 
    * Go into where you install mongodb: ..\MongoDB\Server\3.4\bin
    * Run cmd: mongod --dbpath /path/to/save/data
    * REF: https://blog.csdn.net/hh12211221/article/details/78902596 

### Read form MongoDB

* as for classification and feature vector, they are 1 vs. 1: __if there is a classification module, there must be a feature vector one__ . 

In [72]:
def read_md():
    import pymongo

    connection = pymongo.MongoClient('127.0.0.1',27017)
    tdb = connection.MLTechTransfer
    post = tdb.fgbad 
    for x, y in zip(
        post.find({"application_domain": "image; classification"},{ "module": 1, "_id": 0}), 
        post.find({"application_domain": "image; feature_vector"},{ "module": 1, "_id": 0})
    ):
        print(x, y)
    return

read_md()

{'module': 'imagenet-amoebanet_a_n18_f448-classification'} {'module': 'imagenet-amoebanet_a_n18_f448-feature_vector'}
{'module': 'imagenet-inception_resnet_v2-classification'} {'module': 'imagenet-inception_resnet_v2-feature_vector'}
{'module': 'imagenet-inception_v1-classification'} {'module': 'imagenet-inception_v1-feature_vector'}
{'module': 'imagenet-inception_v2-classification'} {'module': 'imagenet-inception_v2-feature_vector'}
{'module': 'imagenet-inception_v3-classification'} {'module': 'imagenet-inception_v3-feature_vector'}
{'module': 'imagenet-mobilenet_v1_025_128-classification'} {'module': 'imagenet-mobilenet_v1_025_128-feature_vector'}
{'module': 'imagenet-mobilenet_v1_025_160-classification'} {'module': 'imagenet-mobilenet_v1_025_160-feature_vector'}
{'module': 'imagenet-mobilenet_v1_025_192-classification'} {'module': 'imagenet-mobilenet_v1_025_192-feature_vector'}
{'module': 'imagenet-mobilenet_v1_025_224-classification'} {'module': 'imagenet-mobilenet_v1_025_224-featu

In [91]:
def read_from_mongodb(query = {}, projection = {}, order = None, distinct = None):
    import pymongo
    connection = pymongo.MongoClient('127.0.0.1',27017)
    tdb = connection.MLTechTransfer
    post = tdb.fgbad 
    if distinct == None:
        return post.find(query, projection)
    else:
        return post.find(query, projection).distinct(distinct)

In [98]:
from pprint import pprint
for item in list(read_from_mongodb({"application_domain": "image; classification"},{ "family": 1, "family_description": 1, "_id": 0})):
    pprint(item)
    print("\n")

{'family': 'imagenet-amoebanet',
 'family_description': 'AmoebaNet is a family of convolutional neural networks '
                       'for image classification; The architectures of its '
                       'convolutional cells (or layers) have been found by an '
                       'evolutionary architecture search in the NASNet search '
                       'space'}


{'family': 'imagenet-inception_resnet_v2',
 'family_description': 'Inception ResNet V2 is a neural network architecture '
                       'for image classification'}


{'family': 'imagenet-inception_v1',
 'family_description': 'Inception V1 (a.k.a. GoogLeNet) is a neural network '
                       'architecture for image classification'}


{'family': 'imagenet-inception_v2',
 'family_description': 'Inception V2 is a neural network architecture for '
                       'image classification; Inception V2 uses are more '
                       'powerful architecture made possible by the use of